## Exploring housing markers in Canada and British Columbia


Average value of new mortgage loan 

Canada, provinces and CMAs 2012 Q3 to 2020 Q3

In [1]:
%run ./scr/main.py
import plotly.express as px

In [2]:
data = "./data/"

In [34]:
# a function for data cleaning
def data_prep(data_file):
    #Reading the data and making a copy
    df = pd.read_excel(data_file)
    temp = df.copy()

    #Removing empty rows
    temp.dropna(thresh=2, inplace=True)

    #Reseting columns - Changing the column names using the values in the 1st row
    temp_header = temp.iloc[0,]
    temp = temp[1:]
    temp.columns = temp_header

    #Removing all columns whose names are nan
    temp = temp.loc[:, temp.columns.notnull()]

    #Ensuring the indices go from 0 without skipping any number
    temp.reset_index(inplace=True, drop = True)

    #Categorize Geography as Country, Province, and City
    temp['Region/City'] = ""
    temp['Region/City'][temp.index ==0] = 'Country'
    temp['Region/City'][(temp.index < 11) & (temp.index >0)] = 'Province'
    temp['Region/City'][temp.index >= 11] = 'City'

    #Rearrange columns -- bring column 'Region/City' to the 1st column position
    # Access columns
    cols = temp.columns.tolist()
    # Bring last col to front
    cols = cols[-1:] + cols[:-1]
    # Set new column order
    temp = temp[cols] 

    #Converting data type to the right format
    #Isolating yearly and quarterly columns and convert to numeric.
    int_cols = temp.columns.drop(['Region/City', 'Geography'])
    temp[int_cols] = temp[int_cols].apply(pd.to_numeric, errors='coerce')

    return temp

In [4]:
# a function for preparing the data to the right shape
def slice_data(df, level):
    """
    df: data frame with mortgage data
    level: "Province" or "City"
    Extract a subset of df based on level
    Return a dataframe
    """
    try:
        temp = df[df['Region/City']==level]
        temp = pd.melt(temp, id_vars='Geography', value_vars=temp.columns[2:])
        temp.rename(columns = {3:'Time'}, inplace = True)
        return temp
    except KeyError:
        print("Key not found. Make sure that 'level' is in ['Province','City']")


In [5]:
# a function for box or violin graphs
def graph_region(region_df, graph_type: str, title):
    """
    region_df: reshaped data frame object with mortage data
    graph_type: "box", "violin"
    title: title of the graph
    """
    
    plot_dict = {'box': px.box,'violin': px.violin}
        
    try:
        fig = plot_dict[graph_type](region_df, x="Geography", y="value", color = "Geography", points='all')
        fig.update_layout(title = title)
        fig.update_xaxes(tickangle=-45)
        fig.show()
    
    except KeyError:
        print("Key not found. Make sure that 'graph_type' is in ['box','violin']")

# Mortgage loan

In [6]:
data_file = data + "average-value-new-mortgage-loans-ca-prov-cmas-2012-q3-2020-q3-en.xlsx"
df_mortgage = data_prep(data_file)
level = 'Province'
df_mortgage_long = slice_data(df_mortgage, level = level)

In [ ]:
graph_type = 'box'
graph_region(df_mortgage_long, graph_type=graph_type, title = f'{graph_type} plot of mortgage loans by {level}'.upper())

In [ ]:
# bar plot
df_province = df[df['Region/City']=="Province"]
df_province.iloc[:,2:].plot(kind='bar')

In [ ]:
#line plot
fig = px.line(df_mortgage_long, x='Time', y='value', color = 'Geography',
              title = f'Line plot of mortgage loans by {level}'.upper())
fig.update_xaxes(tickangle=-45)
fig.show()

# Delinquency

In [ ]:
data_delinquency = data + "mortgage-delinquency-rate-ca-prov-cmas-2012-q3-2020-q4-en.xlsx"
df_delin = data_prep(data_delinquency)

In [ ]:
level='Province'
graph_type='box'
title = f'{graph_type} plot of delinquency rate by {level}'.upper()
df_delin_long = slice_data(df_delin, level = level)
graph_region(df_delin_long, graph_type=graph_type, title = title)

# Average Income

In [7]:
data_income = data + "real-average-household-income-after-taxes-tenure-2006-2018-en.xlsx"
df_income = data_prep(data_income)

<ipython-input-3-196ac126c045>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Region/City'][temp.index ==0] = 'Country'
<ipython-input-3-196ac126c045>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Region/City'][(temp.index < 11) & (temp.index >0)] = 'Province'
<ipython-input-3-196ac126c045>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Region/City'][temp.index >= 11] = 'City'


In [ ]:
level='Province'
graph_type='box'
title = f'{graph_type} plot of average household income by {level}'.upper()
df_income_long = slice_data(df_income, level = level)
graph_region(df_income_long, graph_type=graph_type, title = title)

In [ ]:
#line plot
fig = px.line(df_income_long, x='Time', y='value', color = 'Geography',
              title = f'Line plot of avergage household income by {level}'.upper())
#fig.update_xaxes(tickangle=-45)
fig.show()

# Combining 2 data sets - mortgage loans and household income

In [ ]:
#set(df['Geography']).difference(df_income['Geography'])
#len(set(df['Geography']).difference(df_income['Geography']))
set(df[df['Region/City']=='Province']['Geography']).difference(df_income[df_income['Region/City']=='Province']['Geography'])

In [8]:
mortgage = df_mortgage.copy()

# change the name of 2 provinces to make the provinces match in 2 datasets
mortgage_replace = {'Newfoundland':'Newfoundland and Labrador', 'Québec': 'Quebec'}
mortgage.replace(mortgage_replace, inplace=True)

# create yearly data for mortgage for some years
select_year = df_income.columns.tolist()
select_year = select_year[select_year.index(2013.0):]
select_year = [str(int(y)) for y in select_year]

# average over 4 quarters to get the yearly value
q = ['Q1', 'Q2', 'Q3', 'Q4']
for y in select_year:
    y1 = [y+j for j in q]
    mortgage[y] = mortgage[y1].mean(axis=1)
    
# select columns in mortgage data
col_mortgage = ['Region/City', 'Geography']+select_year
mortgage13_18 = mortgage[col_mortgage]

In [9]:
col_income = ['Region/City', 'Geography']+[float(y) for y in select_year]
mortgage13_18.columns = col_income

income = df_income.copy()
income13_18 = income[col_income]

In [17]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [18]:
mortgage13_18['Type'] = 'Mortgage'
income13_18['Type'] = 'Income'

In [32]:
df13_18 = pd.concat([mortgage13_18, income13_18])

In [50]:
df13_18.reset_index(inplace=True, drop=True)
df_province = df13_18[df13_18['Region/City']=='Province']

# Mortgage loan and household income

In [57]:
df_province = pd.melt(df_province, id_vars=['Geography','Type'], value_vars=df_province.columns[2:-1])
#df_province.pivot(index = 'Geography', columns = 'Type', values=[2013.0 + i for i in range(6)])

,Geography,Type,variable,value
0,Newfoundland and Labrador,Mortgage,2013.0,188836.75
1,Prince Edward Island,Mortgage,2013.0,136221.00
2,Nova Scotia,Mortgage,2013.0,161341.50
3,New Brunswick,Mortgage,2013.0,126728.50
4,Quebec,Mortgage,2013.0,156146.00
...,...,...,...,...
115,Ontario,Income,2018.0,55200.00
116,Manitoba,Income,2018.0,45300.00
117,Saskatchewan,Income,2018.0,50600.00
118,Alberta,Income,2018.0,63700.00


## Exercise

1. Identify different ways of visualizing data. 
2. What insights do you gain for the study period? 
3. How are these insights different for each province/city?